## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


The cell below contain the library imports needed to read the CSV data files for The Numbers and Rotten Tomatoes.

In [1]:
# Your code here - remember to use markdown cells for comments as well!
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Testing to see if we can bring and API in to supplement our data
import requests
import json
url = 'https://api.themoviedb.org/3/movie/{movie_id}?api_key=<<503f12af247746f5fc6113f4fed1f3a3>>&language=en-US'
TMDB = requests.get(url)

In [3]:
type(TMDB)

requests.models.Response

In [4]:
# Attemp to access an API
TMDB.status_code

401

In [5]:
TMDB_data = TMDB.json()

In [6]:
TMDB_data['title']

KeyError: 'title'

In [7]:
#Unzipping the SQL
from zipfile import ZipFile
# specifying the zip file name
file_name = "./zippedData/im.db.zip"
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
im.db                                          2021-12-20 16:31:38    169443328
Extracting all the files now...
Done!


In [8]:
#Establish the SQL connection
conn = sqlite3.connect('im.db')

In [9]:
# FILES WE WILL USE FOR OUR ANALYSIS
TN_DF = pd.read_csv("./zippedData/tn.movie_budgets.csv.gz")
ROT_TOM_INFO_DF = pd.read_csv("./zippedData/rt.movie_info.tsv.gz", sep="\t")
    
# FILES THAT WE ARE NOT GOING TO USE IN OUR ANALYSIS
ROT_TOM_REV_DF = pd.read_csv("./zippedData/rt.reviews.tsv.gz", sep="\t", encoding='windows-1252')
TMDB_DF = pd.read_csv("./zippedData/tmdb.movies.csv.gz")
BOM_DF = pd.read_csv("./zippedData/bom.movie_gross.csv.gz")

These are the tables that have the information we are going to combine.

In [10]:
cur = conn.cursor()

In [11]:
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
# Fetch the result and store it in table_names
table_names = cur.fetchall()
table_names

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

In [12]:
IMDB_ratings = pd.DataFrame(
    data=cur.execute("""SELECT * FROM movie_ratings;""").fetchall(),
    columns=[x[0] for x in cur.description]
)
IMDB_ratings.shape

(73856, 3)

In [13]:
#This sets the var sql = to the data frame
IMDB = pd.DataFrame(
    data=cur.execute("""SELECT * FROM movie_basics;""").fetchall(),
    columns=[x[0] for x in cur.description]
)

In [14]:
IMDB['start_year'].min()

2010

In [15]:
IMDB_clean = IMDB.copy()

In [16]:
IMDB_clean = IMDB_clean[IMDB['start_year'] >= 2010]

In [17]:
IMDB_clean.shape

(146144, 6)

In [18]:
#I Looked it uo the world wide numbers are actually the total numbers so Im renaming that colomn
TN_DF

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


Below we are cleaning the data by removing the ID column and also the $ sign and commas from the box office numbers. We also are converting the box offices numbers to int in order to make a new column to represent net income.

In [19]:
TN_clean = TN_DF.copy()

In [20]:
TN_clean.drop('id', axis=1, inplace=True)

In [21]:
TN_clean

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...
5777,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [22]:
# Stripping $

TN_clean['production_budget'] = TN_clean['production_budget'].map(lambda x: x.strip('$,'))
TN_clean['domestic_gross'] = TN_clean['domestic_gross'].map(lambda x: x.strip('$,'))
TN_clean['worldwide_gross'] = TN_clean['worldwide_gross'].map(lambda x: x.strip('$,'))

In [23]:
TN_clean

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,"425,000,000","760,507,625","2,776,345,279"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"410,600,000","241,063,875","1,045,663,875"
2,"Jun 7, 2019",Dark Phoenix,"350,000,000","42,762,350","149,762,350"
3,"May 1, 2015",Avengers: Age of Ultron,"330,600,000","459,005,868","1,403,013,963"
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"317,000,000","620,181,382","1,316,721,747"
...,...,...,...,...,...
5777,"Dec 31, 2018",Red 11,"7,000",0,0
5778,"Apr 2, 1999",Following,"6,000","48,482","240,495"
5779,"Jul 13, 2005",Return to the Land of Wonders,"5,000","1,338","1,338"
5780,"Sep 29, 2015",A Plague So Pleasant,"1,400",0,0


In [24]:
TN_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   object
 3   domestic_gross     5782 non-null   object
 4   worldwide_gross    5782 non-null   object
dtypes: object(5)
memory usage: 226.0+ KB


In [25]:
# stripping ',''

TN_clean = TN_clean.apply(lambda x: x.str.replace(',', ''))

In [26]:
TN_clean

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,Dec 18 2009,Avatar,425000000,760507625,2776345279
1,May 20 2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,Jun 7 2019,Dark Phoenix,350000000,42762350,149762350
3,May 1 2015,Avengers: Age of Ultron,330600000,459005868,1403013963
4,Dec 15 2017,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...
5777,Dec 31 2018,Red 11,7000,0,0
5778,Apr 2 1999,Following,6000,48482,240495
5779,Jul 13 2005,Return to the Land of Wonders,5000,1338,1338
5780,Sep 29 2015,A Plague So Pleasant,1400,0,0


In [27]:
TN_clean[['production_budget', 'domestic_gross', 'worldwide_gross']] = TN_clean[['production_budget', 'domestic_gross', 'worldwide_gross']].apply(pd.to_numeric)

In [28]:
TN_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   int64 
 3   domestic_gross     5782 non-null   int64 
 4   worldwide_gross    5782 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 226.0+ KB


In [29]:
TN_clean['Net Income'] = TN_clean['worldwide_gross'] - TN_clean['production_budget']

In [30]:
TN_clean

,release_date,movie,production_budget,domestic_gross,worldwide_gross,Net Income
0,Dec 18 2009,Avatar,425000000,760507625,2776345279,2351345279
1,May 20 2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875
2,Jun 7 2019,Dark Phoenix,350000000,42762350,149762350,-200237650
3,May 1 2015,Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963
4,Dec 15 2017,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747
...,...,...,...,...,...,...
5777,Dec 31 2018,Red 11,7000,0,0,-7000
5778,Apr 2 1999,Following,6000,48482,240495,234495
5779,Jul 13 2005,Return to the Land of Wonders,5000,1338,1338,-3662
5780,Sep 29 2015,A Plague So Pleasant,1400,0,0,-1400


In [31]:
TN_clean['release_date'] = pd.to_datetime(TN_clean['release_date'])

In [32]:
TN_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       5782 non-null   datetime64[ns]
 1   movie              5782 non-null   object        
 2   production_budget  5782 non-null   int64         
 3   domestic_gross     5782 non-null   int64         
 4   worldwide_gross    5782 non-null   int64         
 5   Net Income         5782 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 271.2+ KB


In [33]:
TN_clean.sort_values(by=['Net Income'], ascending=False)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,Net Income
0,2009-12-18,Avatar,425000000,760507625,2776345279,2351345279
42,1997-12-19,Titanic,200000000,659363944,2208208395,2008208395
6,2018-04-27,Avengers: Infinity War,300000000,678815482,2048134200,1748134200
5,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,1747311220
33,2015-06-12,Jurassic World,215000000,652270625,1648854864,1433854864
...,...,...,...,...,...,...
352,2001-04-27,Town & Country,105000000,6712451,10364769,-94635231
341,2019-06-14,Men in Black: International,110000000,3100000,3100000,-106900000
193,2011-03-11,Mars Needs Moms,150000000,21392758,39549758,-110450242
194,2020-12-31,Moonfall,150000000,0,0,-150000000


Next, we are cleaning the IMDB table to remove "N/A" or "Null" values from the "genres", "runtime_minutes", and "original_title" field. We then sorted all movies by "runtime_minutes" (highest to lowest) and removed duplicates where the movie name and year were the same.

In [34]:
IMDB_clean2 = IMDB.dropna(subset = ['genres'])

In [35]:
IMDB_clean2 = IMDB.dropna(subset = ['runtime_minutes'])

In [36]:
IMDB_clean2 = IMDB.dropna(subset = ['original_title'])

In [37]:
IMDB_clean2.sort_values(by='runtime_minutes', ascending = False, inplace=True)
IMDB_clean2

<ipython-input-37-7ea3f4965137>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IMDB_clean2.sort_values(by='runtime_minutes', ascending = False, inplace=True)


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
132389,tt8273150,Logistics,Logistics,2012,51420.0,Documentary
44840,tt2659636,Modern Times Forever,Modern Times Forever,2011,14400.0,Documentary
123467,tt7492094,Nari,Nari,2017,6017.0,Documentary
87264,tt5068890,Hunger!,Hunger!,2015,6000.0,"Documentary,Drama"
88717,tt5136218,London EC1,London EC1,2015,5460.0,"Comedy,Drama,Mystery"
...,...,...,...,...,...,...
146131,tt9915790,Bobbyr Bondhura,Bobbyr Bondhura,2019,NaN,Family
146138,tt9916428,The Secret of China,The Secret of China,2019,NaN,"Adventure,History,War"
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy


In [38]:
IMDB_clean2 = IMDB_clean2.drop_duplicates(subset = ['original_title','start_year'], keep = 'first')
IMDB_clean2

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
132389,tt8273150,Logistics,Logistics,2012,51420.0,Documentary
44840,tt2659636,Modern Times Forever,Modern Times Forever,2011,14400.0,Documentary
123467,tt7492094,Nari,Nari,2017,6017.0,Documentary
87264,tt5068890,Hunger!,Hunger!,2015,6000.0,"Documentary,Drama"
88717,tt5136218,London EC1,London EC1,2015,5460.0,"Comedy,Drama,Mystery"
...,...,...,...,...,...,...
146131,tt9915790,Bobbyr Bondhura,Bobbyr Bondhura,2019,NaN,Family
146138,tt9916428,The Secret of China,The Secret of China,2019,NaN,"Adventure,History,War"
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy


We are now merging the IMDB Clean2 and IMDB Ratings using the movie ID field. Then we will merge that dataframe with the TN_clean file on two conditions - movie name and start year. In order to apply and merge on the start year condition, we will first need to create a column "release year" within the TN_clean dataframe.

In [39]:
IMDB_Merged = pd.merge(IMDB_clean2,IMDB_ratings,on='movie_id', how = 'left')
IMDB_Merged

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt8273150,Logistics,Logistics,2012,51420.0,Documentary,5.0,17.0
1,tt2659636,Modern Times Forever,Modern Times Forever,2011,14400.0,Documentary,6.2,69.0
2,tt7492094,Nari,Nari,2017,6017.0,Documentary,NaN,NaN
3,tt5068890,Hunger!,Hunger!,2015,6000.0,"Documentary,Drama",NaN,NaN
4,tt5136218,London EC1,London EC1,2015,5460.0,"Comedy,Drama,Mystery",NaN,NaN
...,...,...,...,...,...,...,...,...
144454,tt9915790,Bobbyr Bondhura,Bobbyr Bondhura,2019,NaN,Family,NaN,NaN
144455,tt9916428,The Secret of China,The Secret of China,2019,NaN,"Adventure,History,War",NaN,NaN
144456,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary,NaN,NaN
144457,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy,NaN,NaN


In [40]:
TN_clean['release_year'] = pd.DatetimeIndex(TN_DF['release_date']).year
TN_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       5782 non-null   datetime64[ns]
 1   movie              5782 non-null   object        
 2   production_budget  5782 non-null   int64         
 3   domestic_gross     5782 non-null   int64         
 4   worldwide_gross    5782 non-null   int64         
 5   Net Income         5782 non-null   int64         
 6   release_year       5782 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 316.3+ KB


In [41]:
Movies_DF = pd.merge(TN_clean, IMDB_Merged, left_on = ["movie","release_year"], right_on = ["original_title","start_year"], how = "inner")
Movies_DF.sort_values(by='Net Income', ascending= False)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,Net Income,release_year,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
3,2018-04-27,Avengers: Infinity War,300000000,678815482,2048134200,1748134200,2018,tt4154756,Avengers: Infinity War,Avengers: Infinity War,2018,149.0,"Action,Adventure,Sci-Fi",8.5,670926.0
23,2015-06-12,Jurassic World,215000000,652270625,1648854864,1433854864,2015,tt0369610,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",7.0,539338.0
18,2012-05-04,The Avengers,225000000,623279547,1517935897,1292935897,2012,tt0848228,The Avengers,The Avengers,2012,143.0,"Action,Adventure,Sci-Fi",8.1,1183655.0
27,2018-02-16,Black Panther,200000000,700059566,1348258224,1148258224,2018,tt1825683,Black Panther,Black Panther,2018,134.0,"Action,Adventure,Sci-Fi",7.3,516148.0
72,2018-06-22,Jurassic World: Fallen Kingdom,170000000,417719760,1305772799,1135772799,2018,tt4881806,Jurassic World: Fallen Kingdom,Jurassic World: Fallen Kingdom,2018,128.0,"Action,Adventure,Sci-Fi",6.2,219125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,2020-02-21,Call of the Wild,82000000,0,0,-82000000,2020,tt7504726,Call of the Wild,Call of the Wild,2020,NaN,"Adventure,Animation,Family",NaN,NaN
241,2017-12-13,Bright,90000000,0,0,-90000000,2017,tt5519340,Bright,Bright,2017,117.0,"Action,Crime,Fantasy",6.4,147834.0
190,2019-06-14,Men in Black: International,110000000,3100000,3100000,-106900000,2019,tt2283336,Men in Black: International,Men in Black: International,2019,115.0,"Action,Adventure,Comedy",NaN,NaN
118,2011-03-11,Mars Needs Moms,150000000,21392758,39549758,-110450242,2011,tt1305591,Mars Needs Moms,Mars Needs Moms,2011,88.0,"Adventure,Animation,Family",5.4,19823.0


In [42]:
Movies_DF[Movies_DF.isna().any(axis=1)].sort_values(by = 'movie')

,release_date,movie,production_budget,domestic_gross,worldwide_gross,Net Income,release_year,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
1388,2014-12-31,40 Weeks,900000,0,0,-900000,2014,tt3513086,40 Weeks,40 Weeks,2014,111.0,Documentary,NaN,NaN
424,2019-09-20,Ad Astra,49800000,0,0,-49800000,2019,tt2935510,Ad Astra,Ad Astra,2019,NaN,"Adventure,Drama,Mystery",NaN,NaN
682,2014-03-21,Blood Ties,26000000,42472,2923959,-23076041,2014,tt5621080,Blood Ties,Blood Ties,2014,NaN,Documentary,NaN,NaN
255,2020-02-21,Call of the Wild,82000000,0,0,-82000000,2020,tt7504726,Call of the Wild,Call of the Wild,2020,NaN,"Adventure,Animation,Family",NaN,NaN
218,2015-03-13,Cinderella,95000000,201151353,534551353,439551353,2015,tt5089556,Cinderella,Cinderella,2015,113.0,Fantasy,NaN,NaN
878,2019-07-12,Crawl,17000000,0,0,-17000000,2019,tt8364368,Crawl,Crawl,2019,NaN,"Action,Horror,Thriller",NaN,NaN
1062,2012-04-13,Detention,10000000,0,0,-10000000,2012,tt8408152,Detention,Detention,2012,NaN,Horror,NaN,NaN
1285,2016-01-19,Eden,2300000,0,0,-2300000,2016,tt2132321,Eden,Eden,2016,71.0,Drama,NaN,NaN
1008,2019-12-31,Eli,11000000,0,0,-11000000,2019,tt5294518,Eli,Eli,2019,NaN,Horror,NaN,NaN
1467,2015-09-01,Exeter,25000,0,489792,464792,2015,tt4931374,Exeter,Exeter,2015,92.0,None,NaN,NaN
